<a href="https://colab.research.google.com/github/amirmulla/Lottery-Ticket-Hypothesis/blob/main/lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [140]:
###########################
# Mount Google Drive      #
###########################

from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
results_dir = '/content/drive/MyDrive/Deep Learning Project/Results'
model_dir = '/content/drive/MyDrive/Deep Learning Project/Models'

Mounted at /content/drive/


In [141]:
import torch
from torch import nn
import torch.nn.utils.prune as prune
import numpy as np
from torch import optim
import time

from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import helper

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False, download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

In [142]:
# define the NN architecture
class LeNet(nn.Module):
    def __init__(self, int_range=0.5):
        super(LeNet, self).__init__()
        # number of hidden nodes in each layer
        hidden_1 = 300
        hidden_2 = 100
        # linear layer (784 -> hidden_1)
        self.fc1 = nn.Linear(28 * 28, hidden_1)
        # linear layer (n_hidden -> hidden_2)
        self.fc2 = nn.Linear(hidden_1, hidden_2)
        # linear layer (n_hidden -> 10)
        self.fc3 = nn.Linear(hidden_2, 10)
        # dropout layer (p=0.2)
        # dropout prevents overfitting of data
        self.dropout = nn.Dropout(0.2)
        # activation
        self.activation = nn.ReLU()

        # initilize weights
        self.init_weights_list = []
        self.init_weights(int_range)      

        # prune layers
        self.prune_list = []
        self.prune_list.append(self.fc1)
        self.prune_list.append(self.fc2)
        self.prune_list.append(self.fc3)

    def init_weights(self, int_range=0.5):
        self.fc1.weight.data.uniform_(-int_range, int_range)
        self.fc2.weight.data.uniform_(-int_range, int_range)
        self.fc3.weight.data.uniform_(-int_range, int_range)
        self.init_weights_list.append(self.fc1.weight.data.clone())
        self.init_weights_list.append(self.fc2.weight.data.clone())
        self.init_weights_list.append(self.fc3.weight.data.clone())

    def rewind_weight(self):
        self.fc1.weight.data = self.init_weights_list[0].clone()
        self.fc2.weight.data = self.init_weights_list[1].clone()
        self.fc3.weight.data = self.init_weights_list[2].clone()

    def forward(self, x):
        # flatten image input
        x = x.view(-1, 28 * 28)
        # add hidden layer, with relu activation function
        x = self.activation(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add hidden layer, with relu activation function
        x = self.activation(self.fc2(x))
        # add dropout layer
        x = self.dropout(x)
        # add output layer
        x = self.fc3(x)
        return x

# initialize the NN
model = LeNet()
print(model)

LeNet(
  (fc1): Linear(in_features=784, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=100, bias=True)
  (fc3): Linear(in_features=100, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (activation): ReLU()
)


In [143]:
###########################
# Model Evaluation        #
###########################

def EvaluateModel(model, criterion, loader):
    running_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))    

    for data, target in loader:
        data, target = data.cuda(), target.cuda()
        output = model(data)
        loss = criterion(output, target)
        running_loss += loss.item()*data.size(0)
        _, pred = torch.max(output, 1)
        correct = np.squeeze(pred.eq(target.data.view_as(pred)))
        for i in range(len(target)):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

    out_loss = running_loss/len(loader.sampler)
    out_acc = np.sum(class_correct) / np.sum(class_total)

    return out_loss, out_acc

###########################
# Model Training          #
###########################

def TrainAndEvaluateModel(model, model_name, epochs = 20, lr=0.001):      
    model_res_path = results_dir + '/' + model_name + '.pt'
    model_name = model_dir + '/' + model_name + '.pt'
    print(model_res_path)
    
    model.cuda()

    # Define the loss and optimizer
    criterion = nn.CrossEntropyLoss().cuda()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_loss_list = list(0. for i in range(epochs))
    test_loss_list = list(0. for i in range(epochs))
    train_acc_list = list(0. for i in range(epochs))
    test_acc_list = list(0. for i in range(epochs)) 

    test_loss_min = np.Inf

    for epoch in range(epochs):
        t0 = time.time()

        # Train the model
        model.train()
        for images, labels in train_loader:
            images, labels = images.cuda(), labels.cuda()       
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            train_loss, train_acc = EvaluateModel(model, criterion, train_loader)
            test_loss, test_acc = EvaluateModel(model, criterion, test_loader)             

        print('Epoch: {} ({:.2f} seconds) Train Loss: {:.6f} Train Accuracy: {:.6f} Test Loss: {:.6f} Test Accuracy: {:.6f}'
        .format(epoch+1,time.time()-t0, train_loss, train_acc, test_loss, test_acc))
        
        # save model if validation loss has decreased
        if test_loss <= test_loss_min:
          print('Test loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(test_loss_min,test_loss))
          torch.save(model.state_dict(), model_name)
          test_loss_min = test_loss

        train_loss_list[epoch] = train_loss
        test_loss_list[epoch] = test_loss
        train_acc_list[epoch] = train_acc
        test_acc_list[epoch] = test_acc
    
    acc_tesor_dict = {'train_loss': torch.tensor(train_loss_list),
                      'test_loss': torch.tensor(test_loss_list),
                      'train_acc': torch.tensor(train_acc_list),
                      'test_acc': torch.tensor(test_acc_list)}    

    print('Saving...')
    torch.save(acc_tesor_dict, model_res_path)
    print('Saved')

In [144]:
def print_sparsity(model):
  print("Model sparsity: {:.2f}%".format(
          100. * float(torch.sum(model.fc1.weight == 0) + torch.sum(model.fc2.weight == 0) + torch.sum(model.fc3.weight == 0))
        / float(model.fc1.weight.nelement() + model.fc2.weight.nelement() + model.fc3.weight.nelement())))

In [178]:
##############################
# Run                        #
##############################
epochs = 3
lr = 0.01
batch_size = 64
prune_amount = 0.5
rounds = 5

model = LeNet()

print('Train and evaluate lenet model with Pruning:')

for round in range(rounds):
  print('Start prune round: {}'.format(round+1))
  t0 = time.time()
  TrainAndEvaluateModel(model, 'model_prune_lenet', epochs=epochs, lr=lr)
  for layer in model.prune_list:
    prune.l1_unstructured(layer, name='weight', amount=prune_amount)

  print_sparsity(model)
  #model.rewind_weight()
  model.init_weights(int_range=0.1*round)

print(f'lenet model, took {time.time()-t0: .2f} seconds')

Train and evaluate lenet model with Pruning:
Start prune round: 1
/content/drive/MyDrive/Deep Learning Project/Results/model_prune_lenet.pt
Epoch: 1 (20.66 seconds) Train Loss: 0.367367 Train Accuracy: 0.905317 Test Loss: 0.376822 Test Accuracy: 0.904300
Test loss decreased (inf --> 0.376822).  Saving model ...
Epoch: 2 (20.83 seconds) Train Loss: 0.304711 Train Accuracy: 0.923367 Test Loss: 0.318146 Test Accuracy: 0.924100
Test loss decreased (0.376822 --> 0.318146).  Saving model ...
Epoch: 3 (20.86 seconds) Train Loss: 0.275076 Train Accuracy: 0.932383 Test Loss: 0.336131 Test Accuracy: 0.929000
Saving...
Saved
Model global sparsity: 50.00%
Start prune round: 2
/content/drive/MyDrive/Deep Learning Project/Results/model_prune_lenet.pt
Epoch: 1 (21.46 seconds) Train Loss: 0.160595 Train Accuracy: 0.960800 Test Loss: 0.223181 Test Accuracy: 0.955000
Test loss decreased (inf --> 0.223181).  Saving model ...
Epoch: 2 (21.63 seconds) Train Loss: 0.168927 Train Accuracy: 0.953683 Test Loss

In [179]:
  print('Final:')
  t0 = time.time()
  TrainAndEvaluateModel(model, 'model_prune_lenet', epochs=epochs, lr=lr)

Final:
/content/drive/MyDrive/Deep Learning Project/Results/model_prune_lenet.pt
Epoch: 1 (21.83 seconds) Train Loss: 1.872352 Train Accuracy: 0.296117 Test Loss: 1.871366 Test Accuracy: 0.297900
Test loss decreased (inf --> 1.871366).  Saving model ...
Epoch: 2 (21.69 seconds) Train Loss: 1.812236 Train Accuracy: 0.325033 Test Loss: 1.803413 Test Accuracy: 0.333000
Test loss decreased (1.871366 --> 1.803413).  Saving model ...
Epoch: 3 (21.89 seconds) Train Loss: 1.793870 Train Accuracy: 0.341667 Test Loss: 1.786741 Test Accuracy: 0.349700
Test loss decreased (1.803413 --> 1.786741).  Saving model ...
Saving...
Saved


In [ ]:
######################################
# Loading Trained model              #
######################################
criterion = nn.CrossEntropyLoss().cuda()

train_model = LeNet()
train_model.cuda()
model_name = model_dir + '/model_prune_lenet.pt'
train_model.load_state_dict(torch.load(model_name))

train_model.eval()
with torch.no_grad():
  test_loss, test_acc = EvaluateModel(train_model, criterion, test_loader)         
  print('Test Loss: {:.6f} Test Accuracy: {:.6f}'.format(test_loss, test_acc))

In [169]:
#model_parameters = filter(lambda p: p.requires_grad, train_model.parameters())
params = sum([np.prod(p.size()) for p in model.parameters()])
print(params)

266610


In [170]:
print_sparsity(model)

Model global sparsity: 96.88%


In [ ]:
print(list(model.named_buffers()))